In [1]:
from chatbot.backend.services.vector_db.db import VectorDB
from chatbot.backend.services.models.models import vlm
from chatbot.backend.services.models.embedding_model import embedding_model
import requests

/opt/miniconda3/envs/odprt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are n

In [2]:
image_paths = ["archi.png", "34.png"] #"34.png", "43.png", "46.png", "160.png", 
image_paths = [f"chatbot/backend/sample_images/{path}" for path in image_paths]

In [3]:
print("Testing vlm")
image_summaries = vlm.generate_image_summaries(useful_image_paths=image_paths)

Testing vlm


In [4]:
print("Testing embedding model")
dense_embeddings, sparse_embeddings = embedding_model.encode_texts(image_summaries)

Testing embedding model


In [8]:
data = [
    image_paths,
    ["sample_images"] * 2,
    image_summaries,
    dense_embeddings,
    embedding_model.convert_sparse_embeddings(sparse_embeddings),
]

In [9]:
payload = {
    'data' : data
}

In [19]:
url = "http://0.0.0.0:8000/vector-db/insert-documents/"

# Make the POST request to ingest the data
response = requests.post(url, json=payload)

# Check the response
if response.status_code == 200:
    print("Data ingested successfully")
else:
    print(f"Failed to ingest data: {response.status_code}, {response.text}")

Data ingested successfully


In [46]:
url = "http://0.0.0.0:8000/vector-db/image-search/"

In [69]:
data = {
    "query" : "Model Architecture"
}

In [71]:
# Test Hybrid Search
response = requests.post(url, json=data)
if response.status_code == 200:
    print(response.json())
else:
    print(f"Failed to hybrid search data: {response.status_code}, {response.text}")

{'context': 'Doc_id: chatbot/backend/sample_images/archi.png \n Text: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG Workflow involves a user querying a chatbot, which passes the query to a vector database (Milvus) for document ingestion and preprocessing. The database retrieves the most similar chunks/docs and passes them to a Large Language Model (LLM) for generating an answer. The answer is then stored in a Firebase relational database. The Deployment and Scaling section shows users interacting with a load balancer that routes queries to RAG servers. Analytics & Dashboard involves gathering user feedback, analyzing queries, and refining the system based on human analysis. The legend categorizes the architecture components into generative steps, functions/APIs, databases, and servers.\n\nDoc_id: chatbot/backend/sample_images/archi.png \n Text: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG W

In [72]:
response.json()['context']

'Doc_id: chatbot/backend/sample_images/archi.png \n Text: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG Workflow involves a user querying a chatbot, which passes the query to a vector database (Milvus) for document ingestion and preprocessing. The database retrieves the most similar chunks/docs and passes them to a Large Language Model (LLM) for generating an answer. The answer is then stored in a Firebase relational database. The Deployment and Scaling section shows users interacting with a load balancer that routes queries to RAG servers. Analytics & Dashboard involves gathering user feedback, analyzing queries, and refining the system based on human analysis. The legend categorizes the architecture components into generative steps, functions/APIs, databases, and servers.\n\nDoc_id: chatbot/backend/sample_images/archi.png \n Text: The image outlines the architecture of a RAG (Retrieval-Augmented Generation) system. The RAG Workflow invo